In [68]:
from pathlib import Path
from collections import deque, defaultdict
from copy import deepcopy

In [147]:
data = Path('Day20.txt').read_text().splitlines()
data = [line.split(' -> ') for line in data]

# Part 1

In [148]:
modules = defaultdict(lambda : {'module_type': '', 'inputs': [], 'outputs': [], 'input_states': [], 'module_state': False})
modules['broadcaster']['inputs'] = ['button']

for module_name, destinations in data:
    if module_name != 'broadcaster':
        module_type, module_name = module_name[:1], module_name[1:]
    else:
        module_type = 'broadcaster'
    
    modules[module_name]['module_type'] = module_type
    modules[module_name]['outputs'] = destinations.split(', ')
    
    for d in destinations.split(', '):
        modules[d]['inputs'].append(module_name)
        modules[d]['input_states'].append(False)

In [137]:
def pulse_output(input_pulse, origin, module):
    module = modules[module]
    mod_type = module['module_type']

    match mod_type:
        case 'broadcaster':
            return input_pulse
        case '%':
            if input_pulse == True:
                return None
            else:
                if module['module_state']:
                    module['module_state'] = False
                    return False
                else:
                    module['module_state'] = True
                    return True
        case '&':
            module['input_states'][module['inputs'].index(origin)] = input_pulse

            if False in module['input_states']:
                return True
            else:
                return False

In [138]:
def circuit_state(modules):
    state = []
    for module, module_props in modules.items():
        if module_props['module_type'] == '%':
            state.append(module_props['module_state'])
        elif module_props['module_type'] == '&':
            state += module_props['input_states']
    
    return tuple(state)

In [139]:
states = {}
button_presses = 0

high_pulses = 0
low_pulses = 0

for i in range(1000):

    button_presses += 1

    pulses = deque([[False, 'button', 'broadcaster']])

    start_state = circuit_state(modules)

    while pulses:
        input_pulse, origin, module = pulses.popleft()

        if input_pulse:
            high_pulses += 1
        else:
            low_pulses += 1

        output_pulse = pulse_output(input_pulse, origin, module)

        if output_pulse is None:
            continue

        for dest in modules[module]['outputs']:
            pulses.append([output_pulse, module, dest])
    

In [141]:
high_pulses*low_pulses

825896364

# Part 2

In [155]:
backtrack = deque(['rx'])

88